# Just to Get Hyperparameter

In [ ]:
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import pickle
import joblib
import pandas as pd
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from eli5.permutation_importance import get_score_importances
import eli5
from eli5.sklearn import PermutationImportance
from sklearn import cluster

import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour, plot_optimization_history
from optuna.visualization import plot_parallel_coordinate, plot_slice, plot_param_importances
from hyperopt.pyll.base import scope
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from catboost import Pool, CatBoostClassifier
from kaggler.model import AutoLGB
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss
import random

train = pd.read_csv('data/train.csv').drop(['index', 'FLAG_MOBIL'], axis=1).fillna('NAN')
test = pd.read_csv('data/test.csv').drop(['index', 'FLAG_MOBIL'], axis=1).fillna('NAN')
sample_submission = pd.read_csv('data/sample_submission.csv')

# train데이터와 test데이터 변수를 함께 조정하기 위해 병합
merge_data = pd.concat([train, test], axis = 0)

# DAYS_BIRTH
merge_data['DAYS_BIRTH_month']=np.floor((-merge_data['DAYS_BIRTH'])/30)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/30)/12).astype(int)*12)
merge_data['DAYS_BIRTH_week']=np.floor((-merge_data['DAYS_BIRTH'])/7)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/7)/4).astype(int)*4)

# DAYS_EMPLOYED
merge_data['DAYS_EMPLOYED_month']=np.floor((-merge_data['DAYS_EMPLOYED'])/30)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['DAYS_EMPLOYED_week']=np.floor((-merge_data['DAYS_EMPLOYED'])/7)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/7)/4).astype(int)*4)

# before_EMPLOYED
merge_data['before_EMPLOYED']=merge_data['DAYS_BIRTH']-merge_data['DAYS_EMPLOYED']
merge_data['before_EMPLOYED_month']=np.floor((-merge_data['before_EMPLOYED'])/30)-(
    (np.floor((-merge_data['before_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['before_EMPLOYED_week']=np.floor((-merge_data['before_EMPLOYED'])/7)-(
    (np.floor((-merge_data['before_EMPLOYED'])/7)/4).astype(int)*4)

# DAYS_BIRTH / Income
merge_data['DAYS_BIRTH_month/income_total'] = merge_data['DAYS_BIRTH_month'] / merge_data['income_total']
merge_data['DAYS_BIRTH_week/income_total'] = merge_data['DAYS_BIRTH_week'] / merge_data['income_total']

# DAYS_EMPLOYED / Income
merge_data['DAYS_EMPLOYED_month/income_total'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['income_total']
merge_data['DAYS_EMPLOYED_week/income_total'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['income_total']

# before_EMPLOYED / Income
merge_data['before_EMPLOYED/income_total'] = merge_data['before_EMPLOYED'] / merge_data['income_total']
merge_data['before_EMPLOYED_month/income_total'] = merge_data['before_EMPLOYED_month'] / merge_data['income_total']
merge_data['before_EMPLOYED_week/income_total'] = merge_data['before_EMPLOYED_week'] / merge_data['income_total']

# Income / Family
merge_data['income_total/family_size'] = merge_data['income_total'] / merge_data['family_size']

merge_data['child_num/income_total'] = merge_data['child_num'] / merge_data['income_total']
merge_data['family_size/income_total'] = merge_data['family_size'] / merge_data['income_total']
merge_data['DAYS_BIRTH/income_total'] = merge_data['DAYS_BIRTH'] / merge_data['income_total']
merge_data['DAYS_EMPLOYED/income_total'] = merge_data['DAYS_EMPLOYED'] / merge_data['income_total']
merge_data['DAYS_EMPLOYED/DAYS_BIRTH'] =  merge_data['DAYS_EMPLOYED'] / merge_data['DAYS_BIRTH']

# Income skewed-data
merge_data['income_total'] = np.log1p(merge_data['income_total'])
# merge_data['log_income_total'] = np.log(merge_data['income_total'])
# merge_data['sqrt_income_total'] = np.sqrt(merge_data['income_total'])
# merge_data['boxcox_income_total'] = stats.boxcox(merge_data['income_total'])[0]

merge_data = merge_data.fillna(-999)
train = merge_data[merge_data['credit'] != -999]
test = merge_data[merge_data['credit'] == -999]
test.drop('credit', axis = 1, inplace = True)

train_cols = list(train.columns); train_cols.remove('credit'); train_cols.append('credit')
train = train[train_cols]

train_cat = train.copy()
test_cat = test.copy()

for col in train_cat.columns:
    if train_cat[col].dtypes=='object':
        train_cat[col] =  train_cat[col].astype('category')
        test_cat[col] =  test_cat[col].astype('category')
    
print('Category Encoding Completed')

In [ ]:
def base_lgbmodel(train, verbose=True):
    
    train_x = train.drop(['credit'], axis=1)
    train_y = train['credit']
    
    np.random.seed(0)
    lucky_seeds=np.random.randint(1, 10000, 5)
    score_list = []
    
    for i, seed in enumerate(lucky_seeds):

        kfold = StratifiedKFold(n_splits=5, random_state = seed, shuffle = True) # CV 늘려가면서 하기
        cv=np.zeros((train_x.shape[0], 3))

        for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):

            x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
            y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

            lgbmodel = LGBMClassifier(objective='multiclass', n_estimators=10000, random_state=seed)
            lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None) 

            cv[val_idx,:] = lgbmodel.predict_proba(x_val)
        if verbose==True:
            print(f'multi_logloss: {log_loss(train_y, cv):.4f}')
        score_list.append(log_loss(train_y, cv))
    print(f'Average Logloss: {np.mean(score_list):.4f}')
    return np.mean(score_list)

In [ ]:
# income_type, occyp_type, income_total을 이용하여 클러스터링하기 위해 따로 원 핫 인코딩
kmeans_train = train[['occyp_type', 'income_type', 'income_total']]
kmeans_test = test[['occyp_type', 'income_type', 'income_total']]
object_col = []
for col in kmeans_train.columns:
    if kmeans_train[col].dtype == 'object':
        object_col.append(col)
        
enc = OneHotEncoder()
enc.fit(kmeans_train.loc[:,object_col])

train_onehot_df = pd.DataFrame(enc.transform(kmeans_train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
kmeans_train.drop(object_col, axis=1, inplace=True)
kmeans_train = pd.concat([kmeans_train, train_onehot_df], axis=1)

test_onehot_df = pd.DataFrame(enc.transform(kmeans_test.loc[:,object_col]).toarray(),
             columns=enc.get_feature_names(object_col))
kmeans_test.drop(object_col, axis=1, inplace=True)
kmeans_test = pd.concat([kmeans_test, test_onehot_df], axis=1)

base_lgbmodel(train_cat, verbose=False)
# n_clusters를 3부터 10까지 진행하여 하나씩 성능 체크
score_list = {}
k_means_train_total_df = pd.DataFrame()
k_means_test_total_df = pd.DataFrame()
for i in tqdm(range(3, 12)):
    train_cat = train.copy()
    test_cat = test.copy()

    for col in train_cat.columns:
        if train_cat[col].dtypes=='object':
            train_cat[col] =  train_cat[col].astype('category')
            test_cat[col] =  test_cat[col].astype('category')

    # n_cluster를 늘려가며 클러스터링 진행
    k_means_train_df = pd.DataFrame()
    k_means_test_df = pd.DataFrame()
    k_means = cluster.KMeans(n_clusters=i)
    k_means.fit(kmeans_train)
    k_means_train_df = pd.concat([k_means_train_df, pd.DataFrame(k_means.labels_, columns=[f'cluster_{i}'])], axis=1)
    k_means_train_total_df = pd.concat([k_means_train_total_df, pd.DataFrame(k_means.labels_, columns=[f'cluster_{i}'])], axis=1)
    k_means.fit(kmeans_test)
    k_means_test_df = pd.concat([k_means_test_df, pd.DataFrame(k_means.labels_, columns=[f'cluster_{i}'])], axis=1)
    k_means_test_total_df = pd.concat([k_means_test_total_df, pd.DataFrame(k_means.labels_, columns=[f'cluster_{i}'])], axis=1)

    train_cat = pd.concat([train_cat, k_means_train_df], axis=1)
    test_cat = pd.concat([test_cat, k_means_test_df], axis=1)

    # 클러스터링 결과를 category 타입으로 변경
    for col in train_cat.columns:
        if train_cat[col].dtypes=='int32':
            train_cat[col] =  train_cat[col].astype('category')
            test_cat[col] =  test_cat[col].astype('category')
    print(f'cluster: {i}', end=' ')
    score_list[f'cluster_{i}'] = base_lgbmodel(train_cat, verbose=False)

In [ ]:
# 성능이 좋아지는 클러스터링 개수로만 피처 추출

train_cat = train.copy()
test_cat = test.copy()

for col in train_cat.columns:
    if train_cat[col].dtypes=='object':
        train_cat[col] =  train_cat[col].astype('category')
        test_cat[col] =  test_cat[col].astype('category')
        
n = 1 # 성능이 좋아지는 클러스터 개수별로 정렬한 후 앞의 n개만 추출
train_cat = pd.concat([train_cat, k_means_train_total_df.loc[:, sorted(score_list, key=lambda x: score_list[x])[:n]]], axis=1)
test_cat = pd.concat([test_cat, k_means_test_total_df.loc[:, sorted(score_list, key=lambda x: score_list[x])[:n]]], axis=1)

for col in train_cat.columns:
    if train_cat[col].dtypes=='int32':
        train_cat[col] =  train_cat[col].astype('category')
        test_cat[col] =  test_cat[col].astype('category')
        
print(k_means_train_total_df.loc[:, sorted(score_list, key=lambda x: score_list[x])[:n]].columns.tolist())
print(base_lgbmodel(train_cat, verbose=False))

In [ ]:
train_x = train_cat.drop(['credit'], axis=1) # 데이터 나누기
train_y = train_cat['credit']
test_x = test_cat.copy()

train_x_raw = train_x.copy() # 카테고리 인코딩 된 데이터 저장해두기
test_x_raw = test_x.copy()

train_x_oh = train_x_raw.copy()
test_x_oh = test_x_raw.copy()

enc = LabelEncoder()
for col in train_x_oh.columns:
    if (train_x_oh[col].dtype.name=='category'):
        train_x_oh[col] = enc.fit_transform(train_x_oh[col])
        test_x_oh[col] = enc.fit_transform(test_x_oh[col])

train_x = train_x_oh.copy()
test_x = test_x_oh.copy()
        
print('Label Encoding Completed')

# 3. Modeling

## (2) XGBoost

In [ ]:
def xgb_objective(trial: Trial) -> float:
    params_xgb = {
        "random_state": 91373,
        "verbose": None,
        "num_class": 3,
        "objective": "multi:softprob",
        "eval_metric": "mlogloss",
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        "reg_alpha": trial.suggest_uniform("reg_alpha", 0.1, 1.0), # default=0
        "reg_lambda": trial.suggest_uniform("reg_lambda", 0.1, 1.0), # default=1
        "max_depth": trial.suggest_int("max_depth", 8, 100),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.3, 1.0), # default=0
        "colsample_bylevel": trial.suggest_uniform("colsample_bylevel", 0.3, 1.0),
        "subsample": trial.suggest_uniform("subsample", 0.3, 1.0), # default=1
        "gamma": trial.suggest_uniform("gamma", 0.0, 1.0), # default=0
        "min_child_weight": trial.suggest_uniform("min_child_weight", 1, 10), # default=1
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    # CV=10으로 튜닝
    
    seed = 91373
    kfold = StratifiedKFold(n_splits=5, random_state = seed, shuffle = True) # Cross-validation cv=5
    cv = np.zeros((train_x.shape[0], 3))

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):

        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
        dtrain = xgb.DMatrix(x_train, label=y_train)
        dvalid = xgb.DMatrix(x_val, label=y_val)
        watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
                                                                                            # 진행상황 보고싶을때 None을 100으로
        xgbmodel = xgb.train(params_xgb, dtrain, 100000, watchlist, early_stopping_rounds=30, verbose_eval=None)

        cv[val_idx, :] = xgbmodel.predict(dvalid)
        #print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}') # Fold마다 점수 체크하려면 주석 해제
    print('multi_logloss:', log_loss(train_y, cv))

    
    return log_loss(train_y, cv)

In [ ]:
sampler = TPESampler(seed=42)
xgb_study = optuna.create_study(study_name="xgbm_parameter_opt", direction="minimize", sampler=sampler)
xgb_study.optimize(xgb_objective, n_trials=100)

xgb_best_hyperparams = xgb_study.best_trial.params
xgb_base_hyperparams = {'objective':'multi:softprob', "num_class": 3, "eval_metric": "mlogloss", "random_state": 91373}
xgb_best_hyperparams.update(xgb_base_hyperparams)
print("The best hyperparameters are:\n", xgb_best_hyperparams)

In [ ]:
# 결과 나오자마자 복붙해서 카톡방에 올려서 따로 저장해두기!! (파라미터값, logg값 둘 다)

In [ ]:
plot_optimization_history(xgb_study) 

In [ ]:
plot_param_importances(xgb_study)

In [ ]:
plot_importances(xgb_study)

## (4) Catboost

In [ ]:
def cat_objective(trial: Trial) -> float:
    cat_params = {
        'loss_function': 'MultiClass',
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']), # Poisson은 gpu만
        'l2_leaf_reg': trial.suggest_loguniform("l2_leaf_reg", 1e-3, 100),
        'random_strength': trial.suggest_loguniform("random_strength", 1e-2, 100),
        'max_bin': trial.suggest_int('max_bin', 150, 500), # border_counts
        'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.01, 1.0), # rsm # gpu not support
        'learning_rate':trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        'n_estimators':  50000,
        'depth': trial.suggest_int('depth', 5, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100), # min_child_samples
     }
    if cat_params["bootstrap_type"] == "Bayesian":
        cat_params["bagging_temperature"] = trial.suggest_loguniform('bagging_temperature', 1e-2, 100)
    else:
        cat_params["subsample"] = trial.suggest_uniform("subsample", 0.1, 1.0)
        
    
    
    # CV=10으로 튜닝
    
    seed = 91373
    kfold = StratifiedKFold(n_splits=5, random_state = seed, shuffle = True)
    cv = np.zeros((train_x.shape[0], 3))

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):

        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
        catmodel = CatBoostClassifier(**cat_params)                                       # 진행상황 보고싶을때 False를 1000으로
        catmodel.fit(x_train, y_train, eval_set=[(x_val,y_val)], early_stopping_rounds=30, verbose=False)

        cv[val_idx, :] = catmodel.predict_proba(x_val)
        #print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}') # Fold마다 점수 체크하려면 주석 해제
    #print('multi_logloss:', log_loss(train_y, cv))

    
    return log_loss(train_y, cv)

In [ ]:
sampler = TPESampler(seed=42)
cat_study = optuna.create_study(study_name="cat_parameter_opt", direction="minimize", sampler=sampler)
cat_study.optimize(cat_objective, n_trials=100)

cat_best_hyperparams = cat_study.best_trial.params
cat_base_hyperparams = {'loss_function': 'MultiClass', 'n_estimators': 50000}
cat_best_hyperparams.update(cat_base_hyperparams)
print("The best hyperparameters are:\n", cat_best_hyperparams)

In [ ]:
# 결과 나오자마자 복붙해서 카톡방에 올려서 따로 저장해두기!! (파라미터값, logg값 둘 다)

In [ ]:
plot_optimization_history(cat_study) 

In [ ]:
plot_param_importances(cat_study)

In [ ]:
plot_importances(cat_study)